In [1]:
# Dependencies
import os
import csv
import json
import time
import requests
import pandas as pd
import numpy as np
from pprint import pprint
from config import api_key
from citipy import citipy
import matplotlib.pyplot as plt

output_dir = os.path.join("Images","")

In [8]:
## Create latitudes and cities lists variables and initialize them as empty lists
cities = []
# Generate a list of cities using a random function up to 1500 in size for scanning
# Note: larger number of cities can be obtained with larger size, however for this task only about 500 are required
latitude = np.random.uniform(low=-90.000, high=90.000, size=2000)
longitude = np.random.uniform(low=-180.000, high=180.000, size=2000)

latitudes_longitudes = zip(latitude, longitude)
count_cities_500 = 0
for latitude_longitude in latitudes_longitudes:
    city = citipy.nearest_city(latitude_longitude[0], latitude_longitude[1]).city_name
    # If city is not in cities, then append the city to the cities' list. Limit the city list to 500
    if city not in cities:
        if count_cities_500 < 500:
            cities.append(city)
            count_cities_500 += 1
        else:
            break
    else:
        continue
print(f"First five are listed here: {cities[:5]}...")
print(f"There are {len(cities)} cities in the list.")

First five are listed here: ['upernavik', 'punta arenas', 'torbay', 'qandahar', 'ipixuna']...
There are 500 cities in the list.


In [ ]:
# url for weather api
units = 'english'
url = "http://api.openweathermap.org/data/2.5/weather?q="
url2 = "http://api.openweathermap.org/data/2.5/group?id=524901,703448,2643743&units=metric"
city_5_list = cities[:5]
temp_list = []
lat_list = []
for city in cities:
    try:
        query_test_url = f"http://api.openweathermap.org/data/2.5/weather?q={city}&appid={api_key}"
        #print(query_test_url)
        response = requests.get(query_test_url).json()
        #print(response)
        temp_list.append(response["main"]["temp"])
        lat_list.append(response["coord"]["lon"])
        #print(temp_list)
        #print(lat_list)
    except KeyError:
        print(f"City '{city.capitalize()}' was not found.")

City 'Qandahar' was not found.
City 'Dzhusaly' was not found.
City 'Illoqqortoormiut' was not found.
City 'Warqla' was not found.
City 'Karaul' was not found.
City 'Tsihombe' was not found.
City 'Samusu' was not found.
City 'Meyungs' was not found.
City 'Mentok' was not found.
City 'Attawapiskat' was not found.
City 'Temaraia' was not found.


In [ ]:
# README - Your first requirement is 
    # to create a series of scatter plots to showcase the following relationships:
# * Temperature (F) vs. Latitude
# * Humidity (%) vs. Latitude
# * Cloudiness (%) vs. Latitude
# * Wind Speed (mph) vs. Latitude

# Create an starter df
df = pd.DataFrame({'Temperature (F)': })

In [ ]:
temperaturesF = []
latitudes = []
longitudes = []
humiditiesPerc = []
cloudinessPerc = []
windspeedsMPH = []
max_temperatures = []
countries = []
dates = []
city_two = []

# Create data array from cities list
city_weather = []
store_index  = []
store_cityName = []
store_city_not_found = []
count_city_not_found = 0  # count number of cities not found

# transform cities into datframe for the progress bar
cities_df = pd.DataFrame(cities, columns=['City'])

# Run API calls to get the data for scatter plots as defined in parameters above:

# # set initial count quantities for organization
count_one = 0
set_one = 1

# loops for creating dataframe columns
for city in cities:
    
    try:
        url4 = (f"{url}{city}&appid={api_key}")
        print(url4)
        response = requests.get(url4).json()
        temperaturesF.append(response['main']['temp'])
    except Exception:
        print("City not found. Skipping...")

#     cloudinessPerc.append(response['clouds']['all'])
#     countries.append(response['sys']['country'])
#     dates.append(response['dt'])
#     humiditiesPerc.append(response['main']['humidity'])
#     latitudes.append(response['coord']['lat'])
#     longitudes.append(response['coord']['lon'])
#     max_temperatures.append(response['main']['temp_max'])
#     windspeedsMPH.append(response['wind']['speed'])
#     if count_one > 48:
#         count_one = 1
#         set_one += 1
#         city_two.append(city)
#     else:
#         count_one += 1
#         city_two.append(city)
#     print(f"Processing Record {count_one} of Set {set_one} | {city}")
#     except Exception:
# print("City not found. Skipping...")
# print("------------------------------\nData Retrieval Complete\n------------------------------")

In [ ]:
temp_df = pd.DataFrame({'Cities': (cities),
                        'Temperature, DegF': temperaturesF})
temp_df.head()